# Importing required libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import random # library for random number generation

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display_html

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
print('folium installed.')


Solving environment: done

# All requested packages already installed.

Libraries imported.
folium installed.


# Data acquisition and cleaning:

In [2]:
## Data acuisation from wikipedia page
Data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
BS=BeautifulSoup(Data,'lxml')
print(BS.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [3]:
#table = str(BS.table)
table = BS.find('table',{'class':'wikitable sortable'})
#display_html(table,raw=True)

#### Change the Html table to Panda DataFrame

In [4]:
table_rows = table.find_all('tr')  # to read table rows

In [5]:
dataset = []
for row in table_rows:
    dataset.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(dataset, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
print(df.head())
print(df.tail())

  PostalCode           Borough     Neighbourhood
1        M1A      Not assigned      Not assigned
2        M2A      Not assigned      Not assigned
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront
    PostalCode       Borough          Neighbourhood
283        M8Z     Etobicoke              Mimico NW
284        M8Z     Etobicoke     The Queensway West
285        M8Z     Etobicoke  Royal York South West
286        M8Z     Etobicoke         South of Bloor
287        M9Z  Not assigned           Not assigned


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [7]:
df.shape

(287, 3)

#### To Ignore "Not assigned" celles from "Borough" column

In [8]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [9]:
# Reset the index
df1= df.reset_index()
df2=df1[["PostalCode","Borough","Neighbourhood"]]
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [11]:
# Combining the neighbourhoods with same Postalcode
df3 = df2.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df3.reset_index(inplace=True)
df3.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [12]:
# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df3['Neighbourhood'] = np.where(df3['Neighbourhood'] == 'Not assigned',df3['Borough'], df3['Neighbourhood'])

df3.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [13]:
df3.shape

(103, 3)

# Lattitud and Longtiude coordinates of Toronto's Neighborhoods

In [14]:
#### Latitude and Longitude of Toronto's Neighborhoof is find from "http://cocl.us/Geospatial_data"
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging df3 and lat_long tables

In [17]:
lat_lon.rename(columns={'Postal Code':'PostalCode'}, inplace=True) #similirize the 'PostalCode' column name
df4= pd.merge(df3,lat_lon,on='PostalCode')
df4.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
